In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-price aok-align-center'}).find("span", attrs={"class": "a-offscreen"}).string.strip()
    except AttributeError:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
    
    except AttributeError:
        rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+5&sprefix=playsta%2Caps%2C827&ref=nb_sb_ss_ts-doa-p_1_7"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        try:
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        except:
            continue

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_ps5.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation PS5 Console – God of War Ragnarök ...,$559.99,4.8 out of 5 stars,"14,379 ratings",In Stock
1,PlayStation 5 Console CFI-1102A,,4.9 out of 5 stars,"7,343 ratings",
2,PlayStation 5 Pulse 3D Wireless Headset,,4.7 out of 5 stars,"19,758 ratings",Not Available
3,PlayStation 5 Digital Edition,,4.9 out of 5 stars,"7,199 ratings",
4,PlayStation DualSense Wireless Controller - St...,,4.8 out of 5 stars,"75,088 ratings",Only 13 left in stock - order soon
5,WD_BLACK 2TB SN850 NVMe SSD for PS5 Consoles S...,$179.99,4.8 out of 5 stars,"3,133 ratings",In Stock
6,Skautfold: Usurper - PlayStation 5,,Previous page of related Sponsored Products,,Currently unavailable.
7,DualSense Edge Wireless Controller,,4.8 out of 5 stars,"75,088 ratings",In Stock
8,$50 PlayStation Store Gift Card [Digital Code],$50.00,4.7 out of 5 stars,"261,616 ratings",Available now
9,Corsair MP600 PRO LPX 2TB M.2 NVMe PCIe x4 Gen...,$129.99,4.8 out of 5 stars,"2,775 ratings",In Stock
